In [1]:
import glob
import random
import pandas as pd
import numpy as np
import os
import csv

In [11]:
def create_trials_total(version: str):
    """Create trials for the Episodic extinction experiment, using 5 different stimuli for each trial:
        A context, two narrative stimuli (NSs), a conditioned stimulus (CS) and one unconditioned stimulus (US), with an associated sound.
        
        The experiment has 10 different conditions with 40 trials in total, thus each condition has 4 trials.
        In half of the trials, the US is negative, in the other half it is neutral.
        
        On day 2, two conditions are omitted as a control and the remaining 8 conditions are repeated. 
        In half of the repeated trials, the original context is presented, in the other half a new context is presented.
        
        On day 3, all the stimuli are CSs are presented again as a cue. Memory tests are performed using this cue.
        
        As an overview for the conditions:
        condition 1: continued conditioning, negative US, same context
        condition 2: EXT, negative US, same context
        condition 3: continued conditioning, negative US, new context
        condition 4: EXT, negative US, new context
        condition 5: control, negative US
        condition 6: EXT, neutral US, same context
        condition 7: continued conditioning, neutral US, same context
        condition 8: EXT, neutral US, new context
        condition 9: continued conditioning, neutral US, new context
        condition 10: control, neutral US

        
        ToDo: Add "" to every value in the dictionary"""

    # Define the index possibilities of the stimuli
    contexts_i = list(range(56))
    NS_i = list(range(40))
    CS_i = list(range(40))
    US_A_i = list(range(20))
    US_N_i = list(range(20)) 

    # Create lists with all stimulus 
    contexts = sorted(glob.glob('stimuli_files/contexts/*A*'))[:56]
    NS = sorted(glob.glob('stimuli_files/NS/A*'))[:40]
    CS = sorted(glob.glob('stimuli_files/CS/*'))[:40]
    US_A = sorted(glob.glob('stimuli_files/US/negative*'))[:20]
    US_N = sorted(glob.glob('stimuli_files/US/neutral*'))[:20]
    US_sounds_A = sorted(glob.glob('stimuli_files/USsounds/negative*'))[:20]
    US_sounds_N = sorted(glob.glob('stimuli_files/USsounds/neutral*'))[:20]

    # Remove the directory from the values within the lists
    contexts = [stim.split("/")[-1] for stim in contexts]
    NS = [stim.split("/")[-1] for stim in NS]
    CS = [stim.split("/")[-1] for stim in CS]
    US_A = [stim.split("/")[-1] for stim in US_A]
    US_N = [stim.split("/")[-1] for stim in US_N]
    US_sounds_A = [stim.split("/")[-1] for stim in US_sounds_A]
    US_sounds_N = [stim.split("/")[-1] for stim in US_sounds_N]

    # Define the empty data dictionaries for each day
    stim_day1 = {
        'start': [], 'context': [], 'delim': [], 'NS': [], 'delim2': [], 'CS': [], 'delim3': [],
        'US': [], 'delim4': [], 'US_sound': [], 'delim5': [], 'condition': [], 'delim6': [],
        'trial': [], 'delim7': [], 'valence': [], 'end': []
    }

    stim_day2 = {
        'start': [], 'context': [], 'delim': [], 'CS': [], 'delim2': [], 'US': [], 'delim3': [],
        'US_sound': [], 'delim4': [], 'condition': [], 'delim5': [], 'trial': [], 'delim6': [],
        'valence': [], 'end': []
    }

    stim_day3 = {
        'start': [], 'NS': [], 'delim': [], 'CS': [], 'delim2': [], 'US': [], 'delim3': [],
        'US_sound': [], 'delim4': [], 'condition': [], 'delim5': [], 'trial': [], 'delim6': [],
        'valence': [], 'end': []
    }

    # Additional dictionaries without 'start', 'delim', or 'end'
    stim_day1_clean = {key: [] for key in stim_day1 if key not in ['start', 'delim', 'delim2', 'delim3', 'delim4', 'delim5', 'delim6', 'delim7', 'end']}
    stim_day2_clean = {key: [] for key in stim_day2 if key not in ['start', 'delim', 'delim2', 'delim3', 'delim4', 'delim5', 'delim6', 'end']}
    stim_day3_clean = {key: [] for key in stim_day3 if key not in ['start', 'delim', 'delim2', 'delim3', 'delim4', 'delim5', 'delim6', 'end']}

    # Fill the trials for each day using two for loops, to make 10 conditions with 4 trials each
    for i in range(10):  # Repeat 10 times for 40 rows (10 * 4 = 40)
        for j in range(4):  # Each iteration adds 10 rows

            # Randomly select indexes for each stimulus set
            index_C = contexts_i.pop(random.randint(0, len(contexts_i) - 1))
            index_NS = NS_i.pop(random.randint(0, len(NS_i) - 1))
            index_CS = CS_i.pop(random.randint(0, len(CS_i) - 1))
            
            for List, CleanList in zip([stim_day1, stim_day2, stim_day3], [stim_day1_clean, stim_day2_clean, stim_day3_clean]):
                List['start'].append('{"')
                List['delim'].append('", "')
                List['delim2'].append('", "')
                List['delim3'].append('", "')
                List['delim4'].append('", "')
                List['delim5'].append('", "')
                List['delim6'].append('", "')
                List['end'].append('"},')
                if 'delim7' in List:
                    List['delim7'].append('", "')

                List['condition'].append(i + 1)
                List['trial'].append(j + 1)
                CleanList['condition'].append(i + 1)
                CleanList['trial'].append(j + 1)

                if List == stim_day1:
                    List['context'].append(contexts[index_C])
                    List['NS'].append(NS[index_NS])
                    List['CS'].append(CS[index_CS])
                    CleanList['context'].append(contexts[index_C])
                    CleanList['NS'].append(NS[index_NS])
                    CleanList['CS'].append(CS[index_CS])
                if List == stim_day2:
                    List['context'].append(contexts[index_C])
                    List['CS'].append(CS[index_CS])
                    CleanList['context'].append(contexts[index_C])
                    CleanList['CS'].append(CS[index_CS])
                if List == stim_day3:
                    List['NS'].append(NS[index_NS])
                    List['CS'].append(CS[index_CS])
                    CleanList['NS'].append(NS[index_NS])
                    CleanList['CS'].append(CS[index_CS])

            # Alternating between US_A and US_N for each 5 iterations, and add US sounds for day1 and day2
            if i < 5:
                index_USA = US_A_i.pop(random.randint(0, len(US_A_i) - 1))
                for List, CleanList in zip([stim_day1, stim_day2, stim_day3], [stim_day1_clean, stim_day2_clean, stim_day3_clean]):
                    List['US'].append(US_A[index_USA])
                    List['valence'].append(1)
                    List['US_sound'].append(US_sounds_A[index_USA])
                    CleanList['US'].append(US_A[index_USA])
                    CleanList['valence'].append(1)
                    CleanList['US_sound'].append(US_sounds_A[index_USA])
            else:
                index_USN = US_N_i.pop(random.randint(0, len(US_N_i) - 1))
                for List, CleanList in zip([stim_day1, stim_day2, stim_day3], [stim_day1_clean, stim_day2_clean, stim_day3_clean]):
                    List['US'].append(US_N[index_USN])
                    List['valence'].append(2)
                    List['US_sound'].append(US_sounds_N[index_USN])
                    CleanList['US'].append(US_N[index_USN])
                    CleanList['valence'].append(2)
                    CleanList['US_sound'].append(US_sounds_N[index_USN])

            if i % 10 + 1 in [3, 4, 8, 9]:  # If the condition is 3, 4, 8 or 9, change the context on day 2 
                index_C = contexts_i.pop(random.randint(0, len(contexts_i) - 1))
                stim_day2['context'][-1] = contexts[index_C]
                stim_day2_clean['context'][-1] = contexts[index_C]

    # Create the DataFrames and take out the control conditions for day 2
    day1 = pd.DataFrame(stim_day1)
    stim_day2 = pd.DataFrame(stim_day2)
    day2_new = stim_day2[~stim_day2['condition'].isin([5, 10])]
    day2 = pd.DataFrame(day2_new)
    day3 = pd.DataFrame(stim_day3)

    day1_clean = pd.DataFrame(stim_day1_clean)
    stim_day2_clean = pd.DataFrame(stim_day2_clean)
    pd_day2_clean = stim_day2_clean[~stim_day2_clean['condition'].isin([5, 10])]
    day2_clean = pd.DataFrame(pd_day2_clean)
    day3_clean = pd.DataFrame(stim_day3_clean)

    # Save the DataFrames to text and TSV files
    if not os.path.exists('Stimsets/'):
        os.makedirs('Stimsets/')

    if os.path.exists(f'Stimsets/{version}_day1.txt') or os.path.exists(f'Stimsets/{version}_day2.txt') or os.path.exists(f'Stimsets/{version}_day3_Presentation.txt'):
        print(f'{version} already exists, delete it first to overwrite')
    else:
        # Save the combined columns to text files
        day1['combined'] = day1.apply(lambda row: ''.join(row.values.astype(str)), axis=1)
        day2['combined'] = day2.apply(lambda row: ''.join(row.values.astype(str)), axis=1)
        day3['combined'] = day3.apply(lambda row: ''.join(row.values.astype(str)), axis=1)

        with open(f'Stimsets/{version}_day1.txt', 'w') as f:
            f.write('\n'.join(day1['combined']) + '\n')

        with open(f'Stimsets/{version}_day2.txt', 'w') as f:
            f.write('\n'.join(day2['combined']) + '\n')

        with open(f'Stimsets/{version}_day3.txt', 'w') as f:
            f.write('\n'.join(day3['combined']) + '\n')

        # Save the clean DataFrames to TSV files
        day1_clean.to_csv(f'Stimsets/{version}_day1.tsv', sep='\t', index=False)
        day2_clean.to_csv(f'Stimsets/{version}_day2.tsv', sep='\t', index=False)
        day3_clean.to_csv(f'Stimsets/{version}_day3.tsv', sep='\t', index=False)


In [12]:
# Create the trials for 10 different versions, Important: Only create new versions if necessary. It will overwrite the existing files. 

for i in range(10):
    version = "version" + str(i+1)
    create_trials_total(version)

### This was previous version, just for Presentation experiment:

In [4]:
def create_trials_Presentation(version: str):
    """Create trials for the Episodic extinction experiment, using 5 different stimuli for each trial:
        A context, two narrative stimuli (NSs), a conditioned stimulus (CS) and one unconditioned stimulus (US), with an associated sound.
        
        The experiment has 10 different conditions with 40 trials in total, thus each condition has 4 trials.
        In half of the trials, the US is negative, in the other half it is neutral.
        
        On day 2, two conditions are omitted as a control and the remaining 8 conditions are repeated. 
        In half of the repeated trials, the original context is presented, in the other half a new context is presented.
        
        On day 3, all the stimuli are CSs are presented again as a cue. Memory tests are performed using this cue.
        
        As an overview for the conditions:
        condition 1: continued conditioning, negative US, same context
        condition 2: EXT, negative US, same context
        condition 3: continued conditioning, negative US, new context
        condition 4: EXT, negative US, new context
        condition 5: control, negative US
        condition 6: EXT, neutral US, same context
        condition 7: continued conditioning, neutral US, same context
        condition 8: EXT, neutral US, new context
        condition 9: continued conditioning, neutral US, new context
        condition 10: control, neutral US

        
        ToDo: Add "" to every value in the dictionary"""

    # Define the index possibilities of the stimuli
    contexts_i = list(range(56))
    NS_i = list(range(40))
    CS_i = list(range(40))
    US_A_i = list(range(20))
    US_N_i = list(range(20)) 

    # Create lists with all stimulus 
    contexts = sorted(glob.glob('stimuli_files/contexts/*A*'))[:56]
    NS = sorted(glob.glob('stimuli_files/NS/A*'))[:40]
    CS = sorted(glob.glob('stimuli_files/CS/*'))[:40]
    US_A = sorted(glob.glob('stimuli_files/US/negative*'))[:20]
    US_N = sorted(glob.glob('stimuli_files/US/neutral*'))[:20]
    US_sounds_A = sorted(glob.glob('stimuli_files/USsounds/negative*'))[:20]
    US_sounds_N = sorted(glob.glob('stimuli_files/USsounds/neutral*'))[:20]

    # remove the directionary from the values within the lists
    contexts = [stim.split("/")[-1] for stim in contexts]
    NS = [stim.split("/")[-1] for stim in NS]
    CS = [stim.split("/")[-1] for stim in CS]
    US_A = [stim.split("/")[-1] for stim in US_A]
    US_N = [stim.split("/")[-1] for stim in US_N]
    US_sounds_A = [stim.split("/")[-1] for stim in US_sounds_A]
    US_sounds_N = [stim.split("/")[-1] for stim in US_sounds_N]

    # Define the empty data dictionaries for each day
    stim_day1 ={
        'start': [],
                            'context': [],
        'delim': [],
                            'NS': [],
        'delim2': [],
                            'CS': [],
        'delim3': [],
                            'US': [],
        'delim4': [],
                            'US_sound': [],
        'delim5': [],
                            'condition': [],
        'delim6': [],
                            'trial': [],
        'delim7': [],
                            'valence': [],
        'end': []
    }

    stim_day2 ={
        'start': [],
                            'context': [],
        'delim': [],
                            'CS': [],
        'delim2': [],
                            'US': [],
        'delim3': [],
                            'US_sound': [],
        'delim4': [],
                            'condition': [],
        'delim5': [],
                            'trial': [],
        'delim6': [],
                            'valence': [],
        'end': []
    }

    stim_day3 = {
        'start': [],
                            'NS': [],
        'delim': [],
                            'CS': [],
        'delim2': [],
                            'US': [],
        'delim3': [],
                            'US_sound': [],
        'delim4': [],
                            'condition': [],
        'delim5': [],
                            'trial': [],
        'delim6': [],
                            'valence': [],
        'end': []
    }

    # Fill the trials for each day using two for loops, to make 10 conditions with 4 trials each

    for i in range(10):  # Repeat 10 times for 40 rows (10 * 4 = 40)
        for j in range(4):  # Each iteration adds 10 rows

            # Randomly select indexes for each stimulus set
            index_C = contexts_i.pop(random.randint(0, len(contexts_i) - 1))
            index_NS = NS_i.pop(random.randint(0, len(NS_i) - 1))
            index_CS = CS_i.pop(random.randint(0, len(CS_i) - 1))
            
            for List in [stim_day1, stim_day2, stim_day3]: # Add values to each dictionary
                List['start'].append('{"')
                List['delim'].append('", "')
                List['delim2'].append('", "')
                List['delim3'].append('", "')
                List['delim4'].append('", "')
                List['delim5'].append('", "')
                List['delim6'].append('", "')
                List['end'].append('"},')

                List['condition'].append(i +1)  # Counts from 1 to 10 for each iteration to add to the condition parameter
                List['trial'].append(j+1)  # Increases every 10 rows to get 4 lists of rows
                # stim_day1['valence'] = [1 if 'negative' in us else 2 for us in stim_day1['US']]
                # stim_day2['valence'] = [1 if 'negative' in us else 2 for us in stim_day2['US']]
                # stim_day3['valence'] = [1 if 'negative' in us else 2 for us in stim_day3['US']]

                if List == stim_day1: # There is no NS value for day2
                    List['context'].append(contexts[index_C])
                    List['NS'].append(NS[index_NS])
                    List['CS'].append(CS[index_CS])
                    List['delim7'].append('", "')  
                if List == stim_day2: # There is no NS value for day2
                    List['context'].append(contexts[index_C])
                    List['CS'].append(CS[index_CS])
                if List == stim_day3: # There is no context value for day3
                    List['NS'].append(NS[index_NS])
                    List['CS'].append(CS[index_CS])                         
            
            # Alternating between US_A and US_N for each 5 iterations, and add US sounds for day1 and day2
            # Add a value for valence to the list to know if the US is negative (1) or neutral (2)
            if i < 5:
                index_USA = US_A_i.pop(random.randint(0, len(US_A_i) - 1))

                for List in [stim_day1, stim_day2, stim_day3]:
                    List['US'].append(US_A[index_USA])
                    List['valence'].append(1)
                    List['US_sound'].append(US_sounds_A[index_USA])
            else:
                index_USN = US_N_i.pop(random.randint(0, len(US_N_i) - 1))

                for List in [stim_day1, stim_day2, stim_day3]:
                    List['US'].append(US_N[index_USN])
                    List['valence'].append(2)
                    List['US_sound'].append(US_sounds_N[index_USN])

            if i % 10 + 1 in [3, 4, 8, 9]: # If the condition is 3, 4, 8 or 9, change the context on day 2 
                    index_C = contexts_i.pop(random.randint(0, len(contexts_i) - 1))
                    stim_day2['context'][-1] = contexts[index_C]

    # Create the DataFrames and take out the control conditions on day2
    day1 = pd.DataFrame(stim_day1)
    stim_day2 = pd.DataFrame(stim_day2)
    stim_day2_new = stim_day2[~stim_day2['condition'].isin([5, 10])]
    day2 = pd.DataFrame(stim_day2_new)
    day3 = pd.DataFrame(stim_day3)

    # Concatenate all columns into a single string column for day1
    day1['combined'] = day1.apply(lambda row: ''.join(row.values.astype(str)), axis=1)

    # Concatenate all columns into a single string column for day2
    day2['combined'] = day2.apply(lambda row: ''.join(row.values.astype(str)), axis=1)

    # Concatenate all columns into a single string column for day3
    day3['combined'] = day3.apply(lambda row: ''.join(row.values.astype(str)), axis=1)
    # day3
    # version= "version1"

    # make a directory to save the versions in, if it does not exist yet
    if not os.path.exists('Stimsets/EE/'):
        os.makedirs('Stimsets/EE/')

    # Save the DataFrames to text files
    if os.path.exists(f'Stimsets/EE/{version}_day1.txt') or os.path.exists(f'Stimsets/EE/{version}_day2.txt') or os.path.exists(f'Stimsets/EE/{version}_day3.txt'):
        print(f'{version} already exists, delete it first to overwrite')
    else:
        # Save the combined columns to text files
        with open(f'Stimsets/EE/{version}_day1.txt', 'w') as f:
            f.write('\n'.join(day1['combined']) + '\n')

        with open(f'Stimsets/EE/{version}_day2.txt', 'w') as f:
            f.write('\n'.join(day2['combined']) + '\n')

        with open(f'Stimsets/EE/{version}_day3.txt', 'w') as f:
            f.write('\n'.join(day3['combined']) + '\n')